In [2]:
import pandas as pd
import janitor
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

/tmp/ipykernel_59687/2145082868.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv('../data/chembl_tcruzi_percenteffect.csv',sep = ';').clean_names()


In [3]:
raw = pd.read_csv('../data/chembl_tcruzi_percenteffect.csv',sep = ';', low_memory=False).clean_names()

In [49]:
filtered_sm = raw[['smiles','standard_value']].copy().dropna().reset_index(drop = True)
def smiles_to_fp(smiles, radius = 2, fpSize = 2048):
    mol = Chem.MolFromSmiles(smiles)
    generator = Chem.rdFingerprintGenerator.GetMorganGenerator(radius = 2, fpSize = fpSize)
    fp = generator.GetFingerprint(mol)
    return np.array(fp)

def prune_fps(fp_arr):
    """
    remove superfluous bits from fingerprints (if identical in all samples)
    """
    mask = np.all(fp_arr == fp_arr[0,:], axis = 0)
    return fp_arr[:,~mask]

filtered_sm['fp'] = filtered_sm.smiles.apply(lambda x: smiles_to_fp(x, fpSize= 1024))
filtered_sm.to_feather('../data/fps_and_activities.feather')

In [50]:
clean_set = pd.read_feather('../data/fps_and_activities.feather')

In [51]:
clean_set   

,smiles,standard_value,fp
0,O=C(NCCc1ccccc1)NCCN1CCC(c2cccs2)CC1,-81.72,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,O=C(Nc1ccc2c(c1)OCCO2)N1CCCC1c1ccco1,-267.37,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
2,Cc1nc(NC(=O)c2c(-n3cnnn3)sc3c2CCC3)sc1C,-201.83,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CC1=Nc2c(c(=O)[nH]n2C2CCCC2)C(c2cccnc2)S1,-158.05,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Cc1cccc(CSc2ncc(Cl)c(C(=O)O)n2)c1,-152.02,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
72529,O=C(O)C(F)(F)F.O=C(O)C(F)(F)F.c1cc(CN2CCc3ncc(...,-138.40,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
72530,CS(=O)(=O)c1cccc(OCC(=O)N2CCCC2c2ccc[nH]2)c1,-164.37,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
72531,O=C(c1c[nH]c(=O)[nH]c1=O)N1CCCC(c2nccn2CC2CCC2)C1,-178.16,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
72532,OC1COCCN(Cc2cc(-c3ccccc3OC(F)(F)F)no2)C1,-175.77,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
